In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/Files"
print(os.listdir(BASE_DIR))

['train_clean.zip', 'sentinel_images.zip', 'MultimodalFusion.ipynb', 'image_only_model.ipynb', 'data']


In [ ]:
!unzip -o "/content/drive/MyDrive/Files/train_clean.zip" -d "/content/drive/MyDrive/Files"

Archive:  /content/drive/MyDrive/Files/train_clean.zip
  inflating: /content/drive/MyDrive/Files/data/processed/train_clean.csv  


In [ ]:
!unzip -o "/content/drive/MyDrive/Files/sentinel_images.zip" -d "/content/drive/MyDrive/Files/data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5499.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/55.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/550.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5500.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5501.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5502.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5503.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5504.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5505.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5506.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5507.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5508.png  
  inflating: /content/drive/MyDrive/Files/data/images/sentinel/5

In [ ]:
# Setup & Imports
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
class SatellitePriceDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        # Reset index to create a clean 0-based index, but keep the old index
        # which corresponds to the image ID, by not dropping it.
        self.df = df.reset_index()
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Use the 'index' column (which holds the original image ID) for the filename
        img_id = self.df.loc[idx, "index"]
        img_path = os.path.join(self.image_dir, f"{img_id}.png")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # Get the price corresponding to the current positional index in the reset dataframe
        price = torch.tensor(self.df.loc[idx, "price"], dtype=torch.float32)
        return image, price


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
import os

df = pd.read_csv(f"{BASE_DIR}/data/processed/train_clean.csv")

image_dir = f"{BASE_DIR}/data/images/sentinel"

available_images = set(
    int(f.replace(".png", ""))
    for f in os.listdir(image_dir)
    if f.endswith(".png")
)

df_final = df[df.index.isin(available_images)].copy()
# df whose images are downloaded

print(f"Samples with images: {len(df)}")


Samples with images: 16209


In [ ]:
# Load Data & Split
# df = pd.read_csv("data/processed/train_clean.csv")
# image_dir = "data/images/sentinel"

train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42)

train_ds = SatellitePriceDataset(train_df, image_dir, transform)
val_ds   = SatellitePriceDataset(val_df, image_dir, transform)

# train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
# val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

train_loader = DataLoader(
    train_ds,
    batch_size=32,  # smaller batch
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
# CNN Model (ResNet18 → Regression)
class ImageRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.backbone(x).squeeze(1)
model = ImageRegressor().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#Understand

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 213MB/s]


In [ ]:
# TRAINING LOOP (SMALL DATASET)
EPOCHS = 20
PATIENCE = 4  # early stopping

best_rmse = float("inf")
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_losses = []

    for imgs, prices in tqdm(train_loader):
        imgs = imgs.to(device)
        prices = prices.to(device)
        prices_log = torch.log1p(prices)

        optimizer.zero_grad()
        preds_log = model(imgs)
        loss = criterion(preds_log, prices_log)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # VALIDATION
    model.eval()
    val_preds_log, val_true_log = [], []

    with torch.no_grad():
        for imgs, prices in val_loader:
            imgs = imgs.to(device)
            prices = prices.to(device)
            prices_log = torch.log1p(prices)

            preds_log = model(imgs)
            val_preds_log.extend(preds_log.cpu().numpy())
            val_true_log.extend(prices_log.cpu().numpy())

    rmse_log =np.sqrt(mean_squared_error(val_true_log, val_preds_log))
    train_mse = np.mean(train_losses)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train MSE: {train_mse:.2f} | "
        f"Val RMSE: {rmse_log:.2f}"
    )
    # Convert back to real prices (for interpretation only)
    val_preds_real = np.expm1(np.array(val_preds_log))
    val_true_real = np.expm1(np.array(val_true_log))
    rmse_real = np.sqrt(mean_squared_error(val_true_real, val_preds_real))
    print(f"Val RMSE (real prices): {rmse_real:,.0f}")

    # EARLY STOPPING
    if rmse_log < best_rmse:
        best_rmse = rmse_log
        patience_counter = 0
        torch.save(model.state_dict(), "image_model_best.pth")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break


100%|██████████| 406/406 [04:02<00:00,  1.67it/s]


Epoch 1/20 | Train MSE: 22.53 | Val RMSE: 0.46
Val RMSE (real prices): 326,056


100%|██████████| 406/406 [01:47<00:00,  3.78it/s]


Epoch 2/20 | Train MSE: 0.17 | Val RMSE: 0.38
Val RMSE (real prices): 296,331


100%|██████████| 406/406 [01:43<00:00,  3.93it/s]


...

Epoch 13/20 | Train MSE: 0.06 | Val RMSE: 0.43
Val RMSE (real prices): 288,469


100%|██████████| 406/406 [01:43<00:00,  3.94it/s]


Epoch 14/20 | Train MSE: 0.06 | Val RMSE: 0.35
Val RMSE (real prices): 261,161


100%|██████████| 406/406 [01:43<00:00,  3.94it/s]


Epoch 15/20 | Train MSE: 0.05 | Val RMSE: 0.32
Val RMSE (real prices): 239,537
Early stopping triggered.


In [ ]:
import os
BASE_DIR = "/content/drive/MyDrive/Files"
os.makedirs(BASE_DIR, exist_ok=True)

print("Directory exists:", os.path.exists(BASE_DIR))

Directory exists: True


In [ ]:
import torch
torch.save(model.state_dict(), f"{BASE_DIR}/image_only_resnet.pth")

print("Saved files:")
print(os.listdir(BASE_DIR))


Saved files:
['train_clean.zip', 'sentinel_images.zip', 'MultimodalFusion.ipynb', 'image_only_model.ipynb', 'data', 'image_only_resnet.pth']
